# ![HS-Albsig Logo](https://www.hs-albsig.de/typo3conf/ext/hsas/Resources/Public/Images/logo.png)

<br><br><br>
<a class="anchor" id="Anfang"></a>
# Python: Applied Modells - Projektaufgabe

***
**Autoren:** Christian Seidler, Nils Heimbach


**Verlauf:**
Das Notebook wurde im Rahmen der Veranstaltung "Python: Applied Modells" des Studiengangs "Data Engineering and Consulting" der Hochschule Albstadt-Sigmaringen im Sommersemesters 2022 erstellt.
***

***
**Beschreibung:**


**Inhaltsverzeichnis:**
<ul>
<li><a href='#0'>0: Pre-Settings</a></li>
<li><a href='#1'>1: Data Preparation for Regression </a></li>
<li><a href='#2'>2: Metrics for Regression </a></li>  
<li><a href='#3'>3: Train Test Spit </a></li>  
    <li><a href='#4'>4: Models</a></li> 
    <ul>
        <li><a href='#4.1'>4.1: Multiple Linear Regression </a></li>
        <li><a href='#4.2'>4.2: Ridge Regression</a></li> 
        <li><a href='#4.3'>4.3: Lasso Regerssion</a></li> 
        <li><a href='#4.4'>4.4: Elsatic Net</a></li> 
    </ul>
<li><a href='#5'>5: Experimentell: Regression mit einem neuronalen Netzwerk - LSTM</a></li>  
</ul>